# 구글 드라이브 마운트

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


# 라이브러리 설치

In [ ]:
!pip install -qU langchain-text-splitters langchain-community langgraph langchain-huggingface faiss-cpu bitsandbytes accelerate pypdf

# 문서 파일 불러오기

In [ ]:
from langchain_community.document_loaders import PyPDFLoader

file_path = "/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/rag_files/year-end-tax.pdf"
loader = PyPDFLoader(file_path)
pages = []
async for page in loader.alazy_load():
    pages.append(page)

# 문서 청킹

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
)
all_splits = text_splitter.split_documents(pages)

# Vector DB에 저장

In [ ]:
import faiss
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS


# embedding_model_name = "nlpai-lab/KURE-v1"
embedding_model_name = "nlpai-lab/KoE5"

embeddings = HuggingFaceEmbeddings(
    model_name=embedding_model_name,
)

embedding_dim = len(embeddings.embed_query("hello world"))
index = faiss.IndexFlatL2(embedding_dim)

vector_store = FAISS(
    embedding_function=embeddings,
    index=index,
    docstore=InMemoryDocstore(),
    index_to_docstore_id={},
)
ids = vector_store.add_documents(documents=all_splits)

retriever = vector_store.as_retriever()

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


# 모델 및 토크나이저 세팅

In [ ]:
from langchain_huggingface import ChatHuggingFace, HuggingFacePipeline
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
import torch


language_model_name = "nlpai-lab/KULLM3"
# language_model_name = "Bllossom/llama-3.2-Korean-Bllossom-AICA-5B"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.float16,
    llm_int8_enable_fp32_cpu_offload=True,
)
model = AutoModelForCausalLM.from_pretrained(
    language_model_name,
    quantization_config=bnb_config,
    trust_remote_code=True,
)
tokenizer = AutoTokenizer.from_pretrained(language_model_name)

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from transformers import pipeline

llm_pipeline = pipeline(
    model=model,
    tokenizer=tokenizer,
    task="text-generation",
    do_sample=True,
    temperature=0.2,
    repetition_penalty=1.2,
    return_full_text=False,
    max_new_tokens=1000,
)
llm = HuggingFacePipeline(pipeline=llm_pipeline)
chat_model = ChatHuggingFace(llm=llm)

Device set to use cuda:0


# RAG

In [ ]:
from langchain_core.prompts import PromptTemplate


template = """
아래에 주어진 맥락을 이용해 질문에 대해 답변해 줘.
주어진 맥락으로 답변이 어려운 상황이라면, 그냥 모른다고 답하면 되고 억지로 답변을 꾸며 내지 마.
최대한 자세하게 답변해 줘.
반드시 한국어로 답변해야 해.

맥락:
{context}

질문:
{question}
"""

prompt = PromptTemplate.from_template(template)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

def format_docs(docs):
    print(docs)
    return "\n\n".join(doc.page_content for doc in docs)

retrieval_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "연말 정산 때 비거주자가 주의할 점을 알려 줘."

result = retrieval_chain.invoke(question)
print(result)

[Document(id='a9b89cbb-aadb-49f9-ae12-7fcd4fa97fdf', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-22T23:44:00+09:00', 'moddate': '2025-01-09T17:28:20+09:00', 'source': '/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/rag_files/year-end-tax.pdf', 'total_pages': 426, 'page': 102, 'page_label': '103'}, page_content='확정신고기한까지 신고납부한 것으로 본다.(소령 §134④)\n05 비거주자의 연말정산\n가. 거주자와 비거주자\n1) 거주자와 비거주자 (소법 §1의2)\n  거주자는 국내에 주소를 두거나 183일 이상의 거소를 둔 개인을 말하며, 비거주자는 거주자가\n아닌 개인을 말한다.\n2) 주소와 거소의 판정 (소령 §2)\n○ 주소는 국내에서 생계를 같이 하는 가족 및 국내에 소재하는 자산의 유무 등 생활관계의 \n객관적 사실에 따라 판정하며, 거소는 주소지 외의 장소 중 상당기간에 걸쳐 거주하는 \n장소로서 주소와 같이 밀접한 일반적 생활관계가 형성되지 아니한 장소이다.\n○ 국내에 거주하는 개인이 국내에 주소를 가진 것으로 보는 경우\n   - 계속하여 183일 이상 국내에 거주할 것을 통상 필요로 하는 직업을 가진 때\n   - 국내에 생계를 같이하는 가족이 있고, 그 직업 및 자산 상태에 비추어 계속하여 183일 이상 \n국내에 거주할 것으로 인정되는 때'), Document(id='37eac7a8-7d70-4dcb-b136-2f0e07db9539', metadata={'producer': 'ezPDF Builder Su

In [ ]:
question = "2024년 개정 세법 중에 월세와 관련한 내용이 있을까?"

result = retrieval_chain.invoke(question)
print(result)

[Document(id='7cacc321-4c6d-42c1-8e32-d370bd1d636c', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-22T23:44:00+09:00', 'moddate': '2025-01-09T17:28:20+09:00', 'source': '/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/rag_files/year-end-tax.pdf', 'total_pages': 426, 'page': 32, 'page_label': '33'}, page_content='01. 2024년 귀속 연말정산 개정세법 요약\n17\n24 월세액 세액공제 소득기준 및 한도 상향 \n(조세특례제한법 제95조의2, 제122조의3)\n<개정취지> 서민·중산층 주거비 부담 완화\n종          전 개          정\n▢ 월세 세액공제 ▢ 소득기준 및 한도 상향\n○ (대상) 총급여 7천만원(종합소득금액 6천만원) 이하 무\n주택근로자 및 성실사업자 등\n○ 총급여 8천만원(종합소득금액 7천만원) 이하 무주택\n근로자 및 성실사업자 등\n○ (공제율) 월세액의 15% 또는 17%*\n   ＊ 총급여 5,500만원 또는 종합소득금액 4,500만원 이하자\n○ (좌  동)\n○ (공제한도) 연간 월세액 750만원 ○ 750만원 → 1,000만원\n○ (대상 주택) 국민주택규모(85㎡) 이하 또는 기준시가 \n4억원 이하\n○ (좌  동)\n<적용시기> 2024.1.1. 이후 개시하는 과세연도 분부터 적용\n25 신용카드 소득공제율 한시 상향 등'), Document(id='e8ef67ba-c72b-4ee2-bea4-9bc25177951f', metadata={'producer': 'ezPDF Build

In [ ]:
question = "기부금 공제 때 주의할 점은?"

result = retrieval_chain.invoke(question)
print(result)

[Document(id='6fe931b5-3714-4d3a-a412-3e463908b11d', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-22T23:44:00+09:00', 'moddate': '2025-01-09T17:28:20+09:00', 'source': '/content/drive/Shareddrives/스프린트(AI) 드라이브/트랙 Master 폴더/스프린트 미션 및 모범답안/data/rag_files/year-end-tax.pdf', 'total_pages': 426, 'page': 244, 'page_label': '245'}, page_content='본인 800,000 500,000 200,000 100,000\n배우자\n직계비속\n직계존속\n형제자매\n그외\n❹ 기부금 조정 명세\n기부금\n코드\n기부\n연도 \x00\x00 기부금액 ⑰ 전년까지\n공제된 금액\n⑱ 공제대상\n금액(\x00\x00-⑰)\n해당 연도 공제금액 해당 연도에 공제받지 못한 금액\n필요경비 세액(소득)공제 소멸금액 이월금액\n10 2023 500,000 - 500,000 - 500,000\n20 2023 200,000 - 200,000 - 200,000\n43 2023 100,000 100,000 100,000'), Document(id='3edca478-2524-4e6e-a9bd-39242b7b6902', metadata={'producer': 'ezPDF Builder Supreme', 'creator': 'PyPDF', 'creationdate': '2024-12-22T23:44:00+09:00', 'moddate': '2025-01-09T17:28:20+09:00', 'source': '/content/drive/Shareddrives/스프린트(AI) 드라이브/트ᄅ